In [3]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests

In [10]:


url='https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/115.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers)
response.raise_for_status()

html_content = response.text

# Parse all tables and pick the one that contains a Symbol (or Ticker) column.
tables = pd.read_html(html_content)
df_sp = None
for tbl in tables:
    cols = [str(c) for c in tbl.columns]
    if any(c.strip() == 'Symbol' for c in cols) or any('Symbol' in c for c in cols):
        df_sp = tbl
        break
if df_sp is None:
    # fallback: look for common alternatives like 'Ticker'
    for tbl in tables:
        cols = [str(c) for c in tbl.columns]
        if any('Ticker' in c for c in cols):
            df_sp = tbl
            break
if df_sp is None:
    raise KeyError("Could not find a table with a 'Symbol' or 'Ticker' column on the page")

# Normalize column name and convert tickers for yfinance (replace '.' with '-')
symbol_col = next((c for c in df_sp.columns if 'Symbol' in str(c) or 'Ticker' in str(c)), None)
tickers = df_sp[symbol_col].astype(str).str.replace('.', '-', regex=False).tolist()

C:\Users\simin\AppData\Local\Temp\ipykernel_18144\3469017320.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html_content)


In [11]:
df= yf.download(tickers, start='2020-01-01')['Close']

C:\Users\simin\AppData\Local\Temp\ipykernel_18144\532441637.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df= yf.download(tickers, start='2020-01-01')['Close']
[*********************100%***********************]  503 of 503 completed


In [19]:
df_ret = df.pct_change().fillna(0)
df_ret_corr = df_ret.corr()
df_ret_corr

C:\Users\simin\AppData\Local\Temp\ipykernel_18144\612118434.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_ret = df.pct_change().fillna(0)


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,XYZ,YUM,ZBH,ZBRA,ZTS
Ticker,,,,,,,,,,,,,,,,,,,,,
A,1.000000,0.499866,0.381276,0.290337,0.510884,0.365397,0.556347,0.479077,0.560011,0.348873,...,0.524630,0.361043,0.332506,0.303988,0.550448,0.467940,0.404857,0.408767,0.562624,0.575683
AAPL,0.499866,1.000000,0.292551,0.341697,0.428518,0.347490,0.566230,0.599166,0.599724,0.312884,...,0.498960,0.372501,0.355604,0.293033,0.493025,0.507117,0.418288,0.357145,0.543943,0.509235
ABBV,0.381276,0.292551,1.000000,0.033413,0.433202,0.365405,0.344000,0.247705,0.288403,0.292225,...,0.362830,0.199408,0.344163,0.307222,0.315915,0.164722,0.336561,0.349619,0.235013,0.398643
ABNB,0.290337,0.341697,0.033413,1.000000,0.136251,0.105793,0.298397,0.342319,0.382572,0.109059,...,0.215333,0.323935,-0.009784,0.146349,0.272533,0.426717,0.194475,0.182454,0.361728,0.243068
ABT,0.510884,0.428518,0.433202,0.136251,1.000000,0.382735,0.484879,0.402328,0.400421,0.333420,...,0.454468,0.188265,0.478255,0.214142,0.450875,0.336805,0.408656,0.415611,0.368606,0.539429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYZ,0.467940,0.507117,0.164722,0.426717,0.336805,0.257254,0.472048,0.525569,0.516804,0.201263,...,0.442644,0.401122,0.224308,0.212602,0.428359,1.000000,0.347200,0.325115,0.511927,0.424657
YUM,0.404857,0.418288,0.336561,0.194475,0.408656,0.478148,0.475703,0.321790,0.427569,0.371083,...,0.559421,0.439409,0.411531,0.371626,0.488174,0.347200,1.000000,0.496010,0.364368,0.451684
ZBH,0.408767,0.357145,0.349619,0.182454,0.415611,0.440418,0.440678,0.279744,0.402090,0.392404,...,0.495026,0.445872,0.304081,0.410523,0.455449,0.325115,0.496010,1.000000,0.370121,0.413044


In [23]:
pairwise_dist = ((1 - df_ret_corr) / 2).pow(1/2)
pairwise_dist

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,XYZ,YUM,ZBH,ZBRA,ZTS
Ticker,,,,,,,,,,,,,,,,,,,,,
A,0.000000,0.500067,0.556203,0.595677,0.494528,0.563295,0.470985,0.510354,0.469036,0.570582,...,0.487530,0.565224,0.577708,0.589920,0.474105,0.515781,0.545501,0.543706,0.467641,0.460606
AAPL,0.500067,0.000000,0.594747,0.573717,0.534547,0.571187,0.465709,0.447680,0.447368,0.586138,...,0.500520,0.560134,0.567625,0.594545,0.503476,0.496429,0.539311,0.566946,0.477523,0.495361
ABBV,0.556203,0.594747,0.000000,0.695193,0.532353,0.563292,0.572713,0.613308,0.596489,0.594884,...,0.564433,0.632689,0.572642,0.588548,0.584844,0.646250,0.575951,0.570255,0.618460,0.548342
ABNB,0.595677,0.573717,0.695193,0.000000,0.657171,0.668658,0.592285,0.573446,0.555620,0.667436,...,0.626365,0.581406,0.710558,0.653319,0.603103,0.535389,0.634636,0.639354,0.564921,0.615196
ABT,0.494528,0.534547,0.532353,0.657171,0.000000,0.555547,0.507504,0.546659,0.547530,0.577313,...,0.522270,0.637077,0.510757,0.626840,0.523987,0.575845,0.543757,0.540550,0.561869,0.479881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYZ,0.515781,0.496429,0.646250,0.535389,0.575845,0.609404,0.513786,0.487048,0.491526,0.631956,...,0.527900,0.547210,0.622773,0.627454,0.534622,0.000000,0.571314,0.580898,0.494000,0.536350
YUM,0.545501,0.539311,0.575951,0.634636,0.543757,0.510809,0.512004,0.582327,0.534991,0.560766,...,0.469350,0.529429,0.542434,0.560524,0.505878,0.571314,0.000000,0.501991,0.563752,0.523601
ZBH,0.543706,0.566946,0.570255,0.639354,0.540550,0.528953,0.528830,0.600107,0.546768,0.551179,...,0.502481,0.526369,0.589881,0.542898,0.521800,0.580898,0.501991,0.000000,0.561195,0.541736


In [ ]:
def euclidean_profile_distance(D, asset_i, asset_j):
    di = D.loc[asset_i].values
    dj = D.loc[asset_j].values
    
    idx_i = D.index.get_loc(asset_i)
    idx_j = D.index.get_loc(asset_j)
    
    di = np.delete(di, idx_i)
    dj = np.delete(dj, idx_j)

    return np.linalg.norm(di - dj)

assets = pairwise_dist.index
N = len(assets)

profile_dist_matrix = pd.DataFrame(0.0, index=assets, columns=assets)

for i in assets:
    for j in assets:
        if i != j:
            profile_dist_matrix.loc[i,j] = euclidean_profile_distance(pairwise_dist, i, j)

profile_dist_matrix

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,XYZ,YUM,ZBH,ZBRA,ZTS
Ticker,,,,,,,,,,,,,,,,,,,,,
A,0.000000,0.669557,1.598281,2.156599,1.118852,1.512425,0.748477,1.065808,0.978266,1.403505,...,1.822096,1.732904,2.064948,1.913262,1.678617,1.565391,1.458997,1.488844,1.476818,1.385685
AAPL,0.669557,0.000000,1.613046,1.981810,1.160543,1.578177,0.745640,0.719795,0.853696,1.450938,...,1.886744,1.728563,2.043601,1.897138,1.723971,1.511632,1.471013,1.475049,1.466179,1.360233
ABBV,1.598281,1.613046,0.000000,1.649211,0.929952,1.637577,1.897247,1.481637,2.150033,1.017177,...,2.513624,1.744142,1.846251,1.868452,2.294920,1.481906,1.658911,1.498788,1.826223,1.529038
ABNB,2.156599,1.981810,1.649211,0.000000,2.014612,2.651082,2.507495,1.507043,2.379505,2.024247,...,3.158272,2.177684,2.313923,2.278169,2.943289,1.937730,2.289113,2.060613,2.390855,2.146107
ABT,1.118852,1.160543,0.929952,2.014612,0.000000,1.515307,1.381824,1.203297,1.799597,1.201366,...,2.112584,1.679853,1.857519,1.796231,1.923465,1.449328,1.456663,1.389850,1.578890,1.353082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYZ,1.565391,1.511632,1.481906,1.937730,1.449328,1.919772,1.785114,1.496154,1.904568,1.556454,...,1.976278,1.266762,2.196463,1.921097,1.700233,0.000000,1.320709,1.263433,0.883715,1.069145
YUM,1.458997,1.471013,1.658911,2.289113,1.456663,1.720647,1.600055,1.646923,1.788009,1.520620,...,1.143329,1.172421,1.547948,1.424975,1.005435,1.320709,0.000000,0.584957,1.094901,0.802653
ZBH,1.488844,1.475049,1.498788,2.060613,1.389850,1.763946,1.685483,1.549888,1.855477,1.455845,...,1.386587,0.911967,1.684116,1.149973,1.255440,1.263433,0.584957,0.000000,1.133966,0.990850
